In [28]:
from transformers import AutoModel, AutoTokenizer
import pandas as pd
import os
import torch
import spacy
import pickle

In [ ]:
# Load data
DATA_PATH = './data/'
pickle_file = os.path.join(DATA_PATH, 'prepd_data.pkl')
df = pd.read_pickle(pickle_file)

# Load Hebrew stopwords from file
stopwords_file = os.path.join(DATA_PATH, 'stopwords.txt')
with open(stopwords_file, 'r', encoding='utf-8') as file:
    hebrew_stopwords = file.read().splitlines()

# Check if GPU is available and set the device accordingly
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {DEVICE}')

Using device: cuda


In [30]:
# Load DictaBERT-lex model for Hebrew
tokenizer_hebrew = AutoTokenizer.from_pretrained('dicta-il/dictabert-lex')
model_hebrew = AutoModel.from_pretrained('dicta-il/dictabert-lex', trust_remote_code=True)
model_hebrew = model_hebrew.to(DEVICE)
model_hebrew.eval()

BertForLexPrediction(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(128000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, ele

In [31]:
# Load spaCy model for English
nlp_english = spacy.load('en_core_web_sm')

In [32]:
# Stopwords removal for Hebrew
def remove_hebrew_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word not in hebrew_stopwords]
    return ' '.join(filtered_words)

In [47]:
# Lemmatization function for Hebrew and English texts
def lemmatize_text(text, lang):
    if lang == 'he':  # If Hebrew, use DictaBERT-lex

        text = remove_hebrew_stopwords(text) 
        
        lemmatized_result = model_hebrew.predict([text], tokenizer_hebrew)
        
        # Use lemma if available; otherwise, use original token
        lemmatized_text = " ".join([
            lemma if lemma != "[BLANK]" else word
            for word, lemma in lemmatized_result[0]
        ])  
    elif lang == 'en':  # If English, use spaCy
        text = text.lower()
        doc = nlp_english(text)
        lemmatized_text = " ".join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])
    
    else:
        return text 
    return lemmatized_text


In [48]:
df['lemmatized_headline'] = df.apply(lambda row: lemmatize_text(row['headline'], row['lang']), axis=1)
df['lemmatized_text'] = df.apply(lambda row: lemmatize_text(row['combined_info'], row['lang']), axis=1)

In [49]:
df.head()

,book_id,type,unit,headline,combined_info,lang,lemmatized_headline,lemmatized_text
0,241112,ארכיון בן גוריון - מסמך,נאומים ומאמרים,"חזון עתיד מתוקן, חגי ישראל, חזון הנביאים, תנופ...","ד. בן-גוריון י""ח אייר תש""ט 17.5.49אין עמים ...",he,"חזון עתיד תוקן , חג ישראל , חזון נביא , תנופה ...","ד . בן - גוריון י""ח אייר תש""ט 17 . 5 . 49 עם ר..."
1,241170,ארכיון בן גוריון - מסמך,נאומים ומאמרים,"israel and the middle east, חברות ישראל באו""ם,...",For Israel and Middle EastBy David Ben-Gurion ...,en,"israel middle east חברות ישראל באו""ם גלות בר כ...",israel middle eastby david ben gurion prime mi...
2,241180,ארכיון בן גוריון - מסמך,נאומים ומאמרים,חוק שירות הביטחון,"ישיבה ע' ו, 5.9.1949 חוק שירות הבטחון תש""ט-...",he,חוק שירות ביטחון,"ישיבה ע' ו , 5 . 9 . 1949 חוק שירות ביטחון תש""..."
3,241225,ארכיון בן גוריון - מסמך,נאומים ומאמרים,how ambassador Morgenthau saved Palestine jewr...,HOW AMBASSADOR MORGENTHAU SAVED PALESTINE JEWR...,en,ambassador morgenthau save palestine jewry wor...,ambassador morgenthau save palestine jewry wor...
4,241459,ארכיון בן גוריון - מסמך,נאומים ומאמרים,"[ח""א?] במטה המורחב, צה""ל, מערכת סיני, מלחמת סי...","12.12.57 ח""א במטה המורחבאני מודה, שהדברים של מ...",he,"[ ח""א ? ] הטה מורחב , צה""ל , מערכה סיני , מלחמ...","12 . 12 . 57 ח""א הטה מורחב הודה , דבר משה , דב..."


In [50]:
print(df[df['book_id'] == 241180]['lemmatized_text'].item())
print(df[df['book_id'] == 241180]['combined_info'].item())

ישיבה ע' ו , 5 . 9 . 1949 חוק שירות ביטחון תש"ט - 1949 . תשובה שר ביטחון . שר ביטחון ד . בן - גוריון : כנסת נכבד ! הביע הוקרה ועדה כנסת , נענה תביעה ממשלה לא הלכה פגרה קיץ , דן כנסת שלושה חוק : ( 1 ) חוק שירות ביטחון ; ( 2 ) חוק נכה ; ( 3 ) חוק חינוך . לא היי ויכוח רב כנסת , היי כה רציני , חשוב אחראי ויכוח זה . אין פלא . דן עכשיו חוק , שבו תלוי ביטחון , עצם קיום . עניין חייב מאת דן דבר גוף . ויכוח התנהל חוק שירות ביטחון , נותן כבוד כנסת . אפשר הסכים אפשר לא הסכים , דוב רים נאמר אחריות אמונה . אומנם הערה אחד יוצא כלל , אין התרעם כך . נואם אחד , חשב לא די להם דן הצעה , דבר מציע , מציע עצם , מציע קיבל דבר אהבה . פתח ויכוח מין מתווכח ראשון , חבר כנסת י . ריפטין , החרים החזיק דבר י . גלילי , אחרי בא חבר כנסת י . מרידור . העיר דבר הערה אחד , אחר איחד דיבור שאלה הפרינציפיוניות שהתעוררו ויכוח . מתוך רצון הדגיש חשיבות אימון צבאי חייל , הזכיר אפילו מדינה של ניסיון גדול לי , לקח חייל גיוס נתן לו אימון צבאי . חבר ריפטין עשה דבר תורה גדול . עשה ביאור הסבר פסיכואנליטיים , הגיד , הוא יודע אין התכוון 

In [54]:
df["document"] = "headline: " + df["headline"] + ", text: " + df["combined_info"]
df["lemmatized_document"] = "headline: " + df["lemmatized_headline"] + ", text: " + df["lemmatized_text"]

In [55]:
df.head()

,book_id,type,unit,headline,combined_info,lang,lemmatized_headline,lemmatized_text,document,lemmatized_document
0,241112,ארכיון בן גוריון - מסמך,נאומים ומאמרים,"חזון עתיד מתוקן, חגי ישראל, חזון הנביאים, תנופ...","ד. בן-גוריון י""ח אייר תש""ט 17.5.49אין עמים ...",he,"חזון עתיד תוקן , חג ישראל , חזון נביא , תנופה ...","ד . בן - גוריון י""ח אייר תש""ט 17 . 5 . 49 עם ר...","headline: חזון עתיד מתוקן, חגי ישראל, חזון הנב...","headline: חזון עתיד תוקן , חג ישראל , חזון נבי..."
1,241170,ארכיון בן גוריון - מסמך,נאומים ומאמרים,"israel and the middle east, חברות ישראל באו""ם,...",For Israel and Middle EastBy David Ben-Gurion ...,en,"israel middle east חברות ישראל באו""ם גלות בר כ...",israel middle eastby david ben gurion prime mi...,"headline: israel and the middle east, חברות יש...","headline: israel middle east חברות ישראל באו""ם..."
2,241180,ארכיון בן גוריון - מסמך,נאומים ומאמרים,חוק שירות הביטחון,"ישיבה ע' ו, 5.9.1949 חוק שירות הבטחון תש""ט-...",he,חוק שירות ביטחון,"ישיבה ע' ו , 5 . 9 . 1949 חוק שירות ביטחון תש""...","headline: חוק שירות הביטחון, text: ישיבה ע' ו,...","headline: חוק שירות ביטחון, text: ישיבה ע' ו ,..."
3,241225,ארכיון בן גוריון - מסמך,נאומים ומאמרים,how ambassador Morgenthau saved Palestine jewr...,HOW AMBASSADOR MORGENTHAU SAVED PALESTINE JEWR...,en,ambassador morgenthau save palestine jewry wor...,ambassador morgenthau save palestine jewry wor...,headline: how ambassador Morgenthau saved Pale...,headline: ambassador morgenthau save palestine...
4,241459,ארכיון בן גוריון - מסמך,נאומים ומאמרים,"[ח""א?] במטה המורחב, צה""ל, מערכת סיני, מלחמת סי...","12.12.57 ח""א במטה המורחבאני מודה, שהדברים של מ...",he,"[ ח""א ? ] הטה מורחב , צה""ל , מערכה סיני , מלחמ...","12 . 12 . 57 ח""א הטה מורחב הודה , דבר משה , דב...","headline: [ח""א?] במטה המורחב, צה""ל, מערכת סיני...","headline: [ ח""א ? ] הטה מורחב , צה""ל , מערכה ס..."


In [56]:
with open(pickle_file, "wb") as file: 
    pickle.dump(df, file)